In [1]:
# Add client package to sys path

CLIENT_PATH = '/home/jovyan/client'

import sys

if CLIENT_PATH not in sys.path:
    sys.path.append(CLIENT_PATH)

In [2]:
%load_ext autoreload
%autoreload 2

# Sample dataset

In [3]:
import pandas as pd

In [4]:
schema = {
    'Features': [
        {
            'FeatureName': 'target',
            'FeatureType': 'float'
        },
        {
            'FeatureName': 'group_id_0',
            'FeatureType': 'string'
        },
        {
            'FeatureName': 'group_id_1',
            'FeatureType': 'string'
        },
        {
            'FeatureName': 'timestamp',
            'FeatureType': 'timestamp'
        }
    ]
}

In [5]:
df = pd.read_csv('data/target.csv')
df = df.rename(
    columns={
        'store': 'group_id_0',
        'item': 'group_id_1', 
        'date': 'timestamp', 
        'sales': 'target'
    }
)

In [7]:
dtypes = {
    'target': float,
    'group_id_0': str,
    'group_id_1': str,
}
df = df.astype(dtypes)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Create session instance

In [3]:
# The `session` instance allows us to create clients for any available service.

from client.session import Session

session = Session()

In [4]:
# List available services

session.get_available_services()

['minio', 'forecast']

# Minio client

Minio client allows to write datasets in cloud buckets for later use with the ***forecast*** service.

In [5]:
minio_client = session.create_client(
    service_name='minio', 
    is_secure=False, 
    access_key_id='oxxo', 
    secret_access_key='password'
)

In [18]:
# Write dataset to minio bucket.

minio_client.create_dataset(df, schema, dataset_type='target', name='sample')

# Forecast client

In [15]:
forecast_client = session.create_client(
    service_name='forecast', 
    access_key_id='oxxo', 
    secret_access_key='password'
)

In [16]:
predictor_config = {
    'name': 'sample',
    'algorithm': 'seq2seq',
    'forecast_horizon': 10,
    'perform_hpo': False,
    'bucket_name': 'sample',
    'minio_host': minio_client.endpoint.host,
}

http = forecast_client.create_predictor(**predictor_config)

In [17]:
http

{'service': {'name': 'sample',
  'algorithm': 'seq2seq',
  'perform_hpo': False,
  'bucket_name': 'sample',
  'minio_host': 'http://minio:9000/'},
 'credentials': {'access_key': 'oxxo',
  'secret_key': 'password',
  'token': None}}